In [1]:
import os

from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoTokenizer,\
AutoModelForTokenClassification, AutoModelForMaskedLM, TrainingArguments, Trainer,AutoModelForSequenceClassification,DataCollatorWithPadding
import evaluate
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset,Dataset, DatasetDict


f:\Users\mario\.conda\envs\parking\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load model
model_checkpoint = "mor40/BulBERT-chitanka-model"
model_raw = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mor40/BulBERT-chitanka-model and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
#@title Load Dataset
hf_dataset = load_dataset("bgglue/bgglue","xnlibg")
hf_dataset

Extracting data files: 100%|██████████| 2/2 [00:16<00:00,  8.18s/it]
Generating train split: 392702 examples [00:29, 13538.18 examples/s]
Generating test split: 5010 examples [00:01, 3523.21 examples/s] 
Generating validation split: 2490 examples [00:00, 3378.55 examples/s] 


DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 392702
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5010
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 2490
    })
})